<a href="https://colab.research.google.com/github/k1rh4/2018s-onion-team6/blob/master/ACDC_Email_Bot_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 참가자 공격 플레이북
이 워크플로우를 따라 이메일 프롬프트 인젝션 CTF 서비스와 상호 작용하세요. 각 플레이어는 노트북 사본을 사용하여 탐색하고, 페이로드를 준비하고, 익스플로잇을 확인합니다.

## 1. Configure Target
Set `BASE_URL` to the challenge endpoint assigned by the organisers (include the port if provided).


In [ ]:
BASE_URL = "http://211.115.110.155:8000"


### 1a. API Key & Optional Auth
Insert your personal `X-CTF-API-Key`.


In [ ]:
CTF_API_KEY = "paste-your-api-key"  # required

DEFAULT_HEADERS = {"User-Agent": "rag-ctf-client/1.0"}
DEFAULT_HEADERS["X-CTF-API-Key"] = CTF_API_KEY

## 2. Helper Utilities
The helpers below keep interactions consistent and add basic error handling.

In [ ]:
import requests
from pprint import pprint

session = requests.Session()
session.headers.update(DEFAULT_HEADERS)

def api_get(path: str, **kwargs):
    resp = session.get(f"{BASE_URL}{path}", **kwargs)
    resp.raise_for_status()
    return resp.json()

def api_post(path: str, *, json=None, **kwargs):
    resp = session.post(f"{BASE_URL}{path}", json=json, **kwargs)
    resp.raise_for_status()
    if "application/json" in resp.headers.get("content-type", ""):
        return resp.json()
    return resp.text

def show(title: str, payload):
    print(f"\n### {title}")
    pprint(payload)


## 3. Reconnaissance
Use the helper calls below to confirm the service is reachable and to inspect the documents currently indexed for your team (remember the per-minute quota!).


In [ ]:
docs = api_get("/documents")

show("Documents", docs)


## 4. Stage the Email Payload
Craft a benign-looking email while hiding the malicious instruction in an attachment. The guardrail scans the visible body, so keep it innocuous and let the attachment contain the override.


In [ ]:
from pathlib import Path
import base64

attachment_path = Path()  # update if you use a custom payload
attachment_b64 = base64.b64encode(attachment_path.read_bytes()).decode()

email_payload = {
    "sender": "attacker@example.com",
    "subject": "Bridge call follow-up",
    "body": (
        "Hi Sentinel Support,\n\n"
    ),
    "top_k": 3,
    "metadata": {"thread": "bridge-call"},
    "attachments": [
        {
            "filename": attachment_path.name,
            "content_base64": attachment_b64,
            "content_type": "application/pdf"
        }
    ],
}


### Deliver the phishing email
Post the payload to `/mailbox/send` and inspect the response. You may need to iterate on the attachment contents before the flag appears in `agent_response`.


In [ ]:
email_result = api_post("/mailbox/send", json=email_payload)
print(email_result["agent_response"])
show("Retrieved documents", email_result["retrieved"])
email_result


### Optional: Inspect archived emails
Fetch `/documents` again to confirm your email was ingested and to review the documents that influenced model behaviour.


In [ ]:
docs_after = api_get("/documents")
show("Documents after email", docs_after)

## 5. Reset & Hygiene
Only call `/reset` when working locally or if the organisers explicitly allow it. Resetting clears your team corpus back to the base state.


In [ ]:
# Uncomment the lines below if you have permission to reset your corpus.
# reset_response = api_post("/reset")
# show("Reset", reset_response)


## 6. Notes
- Each API key is limited (default: 5 requests/minute); batch your recon and payload tests.
- A successful exploit will reveal `ACDC{` inside `agent_response`.
- Keep payload variants handy so you can iterate quickly during the event.
- Respect the event rules and avoid impacting other teams.
